# 1) **Necessary Libraries**

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [4]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# 2) **Data Loading**

In [5]:
# Data Loading

def load_data(file):
    data = pd.read_csv(file)
    data = data[:5000]
    return data

In [6]:
# Columns Finding

def finding_num_cat_columns(data):
    data.drop_duplicates(inplace=True)

    numerical_columns = []
    categorical_columns = []

    for i in data.dtypes.index:
        if data.dtypes[i] == 'int64' or data.dtypes[i] == 'float64':
            numerical_columns.append(i)
        elif data.dtypes[i] == 'object':
            categorical_columns.append(i)

    columns = {
        'numerical_columns': numerical_columns,
        'categorical_columns': categorical_columns,
    }
    return columns

# 3) **Target column**

In [7]:
def find_target_column(data):
    numeric_cols = data.select_dtypes(include=['int64', 'float64', 'number'])
    correlations = numeric_cols.corr()
    avg_corr = correlations.mean().sort_values(ascending=False)
    threshold = max(avg_corr)
    target_col = next((col for col, corr in avg_corr.items() if corr >= threshold), None)
    return target_col

# 4) **Preprocessing**

In [8]:
def preprocessing(data, columns, target=None):
    categorical_columns = [col for col in columns['categorical_columns'] if col != target]

    # Numerical pipeline
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # Categorical pipeline
    cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_pipeline, columns['numerical_columns']),
            ('cat', cat_pipeline, categorical_columns)
        ],
        remainder='drop'
    )

    processed_data = preprocessor.fit_transform(data)
    column_names = columns['numerical_columns'] + categorical_columns
    processed_df = pd.DataFrame(processed_data, columns=column_names, index=data.index)

    if target is not None and target in data.columns:
        processed_df[target] = data[target]
    else:
        target = find_target_column(processed_df)
    return processed_df, target


# 5) **Outlier Finding Methods**

In [9]:
# Outlier Finding Methods

def outliersFinding_ZScore(data, threshold=3):
    outliers = pd.DataFrame()

    for i in data.columns:
        mean = np.mean(data[i])
        std = np.std(data[i])
        z_scores = (data[i] - mean) / std
        column_outliers = data[(z_scores > threshold) | (z_scores < -threshold)]
        column_outliers.drop_duplicates(inplace=True)

        outliers = pd.concat([outliers, column_outliers])

    outliers = outliers.drop_duplicates()
    return outliers

# Finding outliers using IQR method
def outliersFinding_IQR(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1

    outliers = (data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))
    return data[outliers.any(axis=1)]

# 6) **Removing Outliers**

In [10]:
# Outliers Removal

def Remove_Outliers(data, outliers_info, threshold=3, strategy='capping'):
    for column, (method, outliers) in outliers_info.items():
        if strategy.lower() == 'capping':
            if method.lower() == 'zscore':
                # Method for removing outliers with z-score and capping
                upper_bound = data[column].mean() + threshold * data[column].std()
                lower_bound = data[column].mean() - threshold * data[column].std()
                data[column] = np.where(data[column] > upper_bound, upper_bound,
                                        np.where(data[column] < lower_bound, lower_bound, data[column]))

            elif method.lower() == 'iqr':
                # Method for capping based on IQR
                Q1 = data[column].quantile(0.25)
                Q3 = data[column].quantile(0.75)
                IQR = Q3 - Q1
                upper_bound = Q3 + 1.5 * IQR
                lower_bound = Q1 - 1.5 * IQR
                data[column] = np.where(data[column] > upper_bound, upper_bound,
                                        np.where(data[column] < lower_bound, lower_bound, data[column]))

        elif strategy.lower() == 'trimming':
            if method.lower() == 'zscore':
                # Method for trimming using Z-score
                upper_bound = data[column].mean() + threshold * data[column].std()
                lower_bound = data[column].mean() - threshold * data[column].std()
                data[column] = data[column].where((data[column] >= lower_bound) & (data[column] <= upper_bound))

            elif method.lower() == 'iqr':
                # Method for trimming using IQR
                Q1 = data[column].quantile(0.25)
                Q3 = data[column].quantile(0.75)
                IQR = Q3 - Q1
                upper_bound = Q3 + 1.5 * IQR
                lower_bound = Q1 - 1.5 * IQR
                data[column] = data[column].where((data[column] >= lower_bound) & (data[column] <= upper_bound))

            # Drop NaNs only if there are no valid entries
            data[column] = data[column].fillna(data[column].median())  # Filling NaNs with median or use another strategy if needed

    return data

# 7) **Outliers Detection**

In [11]:
"""# **Outliers Detection**"""

def Detect_Outliers(data, threshold):
    outliers_info = {}  # Store information about outliers

    for column in data.select_dtypes(include=['number']).columns:

        skewness = data[column].skew()

        if abs(skewness) < 0.5:
            column_outliers = outliersFinding_ZScore(data[[column]], threshold)
            outliers_info[column] = ('zscore', column_outliers)
        else:
            column_outliers = outliersFinding_IQR(data[[column]])
            outliers_info[column] = ('iqr', column_outliers)

    return outliers_info

# 8) **Data Visualization**

In [12]:
# Graph Plotting

def graph_plot(data):
    for col in data.columns:
        plt.figure(figsize=(20, 5))

        plt.subplot(1, 2, 1)
        sns.boxplot(data[col])
        plt.title(f'Boxplot: {col}')
        plt.xticks(rotation=90)

        plt.subplot(1, 2, 2)
        sns.histplot(data[col], kde=True)
        plt.title(f'Histogram: {col}')
        plt.xticks(rotation=90)

        st.pyplot(plt)

# 9) **Model Vadiation**

In [30]:
# Model Cross-Validation

def model_cross_validation(X_train, y_train, model_name):
    model_params = {
        "Decision Tree": {
            "model": DecisionTreeRegressor(),
            "param_grid": {
                "max_depth": [None, 5, 10, 15],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
            },
        },
        "Random Forest": {
            "model": RandomForestRegressor(),
            "param_grid": {
                "n_estimators": [50, 100, 150],
                "max_depth": [None, 5, 10, 15],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
            },
        },
        "Linear Regression": {
            "model": LinearRegression(),
            "param_grid": {
                "fit_intercept": [True, False],
            },
        },
        "SVR": {
            "model": SVR(),
            "param_grid": {
                "C": [0.1, 1, 10],
                "kernel": ["linear", "rbf"],
                "epsilon": [0.1, 0.2, 0.5],
            },
        },
        "AdaBoost": {
            "model": AdaBoostRegressor(),
            "param_grid": {
                "n_estimators": [50, 100, 150],
                "learning_rate": [0.01, 0.1, 1],
            },
        },
    }

    if model_name not in model_params:
        raise ValueError(f"Model '{model_name}' is not supported. Available models: {list(model_params.keys())}")

    config = model_params[model_name]
    random_search = RandomizedSearchCV(
        estimator=config["model"],
        param_distributions=config["param_grid"],
        cv=10,
        n_iter=10,
        scoring=["neg_mean_squared_error","r2"],
        refit="r2",
        n_jobs=-1,
        random_state=42,
        verbose=1,
    )

    random_search.fit(X_train, y_train)
    return random_search

# 10) **Parameter Tuning**

In [33]:
# Check Parameters

def check_parameters(data, model_name, target=None):
    if target not in data.columns:
        raise ValueError(f"Target column '{target}' not found in the dataset.")

    X = data.drop(columns=[target])
    y = data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_results = model_cross_validation(X_train, y_train, model_name)

    return {
        "best_params": model_results.best_params_,
        "best_score": model_results.best_score_,
    }

In [21]:
data = load_data('housing.csv')
columns = finding_num_cat_columns(data)
outliers_info = Detect_Outliers(data, threshold=3)
processed_data = Remove_Outliers(data, outliers_info, threshold=3, strategy='capping')
processed_data, target = preprocessing(processed_data, columns, target='Median_House_Value')

In [41]:
rfr = check_parameters(processed_data, 'AdaBoost', target)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


In [42]:
rfr

{'best_params': {'n_estimators': 50, 'learning_rate': 1},
 'best_score': 0.8988960729356522}

# 11) **Frontend Part**

In [17]:
# Cool visualization function
def show_intro_animation():
    """
    Show a cool animated sine wave plot to engage users with enhanced visuals.
    """
    st.markdown(
        """
        <style>
            @import url('https://fonts.googleapis.com/css2?family=Kanit:ital,wght@0,100;0,200;0,300;0,400;0,500;0,600;0,700;0,800;0,900&display=swap');
            body {
                background-color: #F6F8FC;
            }
            .title {
                font-family: "Kanit", sans-serif;
                font-weight: 900;
                font-size: 85px;
                color: #4A90E2;
                text-align: center;
                margin-left: 20px;
                margin-bottom: 20px;
                animation: slideDown 1s ease-out;
            }
            .subtitle {
                font-family: "Kanit", sans-serif;
                font-size: 18px;
                color: #666666;
                padding: 0 20% 0 20%;
                text-align: center;
                margin-top: -10px;
                animation: fadeIn 1.5s ease-in;
            }
            .centered-container {
                display: flex;
                flex-direction: column;
                align-items: center;
                justify-content: center;
                margin-top: 2.5%;
                margin-bottom: 0;
                height: calc(80vh - 100px);
            }
            .button-container {
                margin-top: 40px;
                display: flex;
                justify-content: center;
            }
            .button-container button {
                font-family: "Kanit", sans-serif;
                background-color: #6C63FF;
                color: #fff;
                font-size: 18px;
                padding: 10px 20px;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                transition: background-color 0.3s ease;
            }
            .button-container button:hover {
                background-color: #574B90;
            }

            @keyframes slideDown {
                from {
                    transform: translateY(-50px);
                    opacity: 0;
                }
                to {
                    transform: translateY(0);
                    opacity: 1;
                }
            }

            @keyframes fadeIn {
                from {
                    opacity: 0;
                }
                to {
                    opacity: 1;
                }
            }
        </style>
        <div class="centered-container">
            <div class="title">Welcome to Optimal Predictor</div>
            <div class="subtitle">
                Unlock the power of your data! Upload your dataset to experience next-level predictions
                and actionable insights. Let Optimal Predictor pave the way to smarter decisions.
            </div>
        </div>
        """,
        unsafe_allow_html=True,
    )
